<a href="https://colab.research.google.com/github/HanNayeoniee/DeepLearning-Study/blob/master/Tensorflow_Object_Detection_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mount your google drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Copy Object detection folders to colab environment

In [0]:
!cp '/content/drive/My Drive/CardID-project/card_detection/object_detection.zip' object_detection.zip

In [0]:
!cp '/content/drive/My Drive/CardID-project/card_detection/slim.zip' slim.zip

In [0]:
!unzip object_detection.zip
!unzip slim.zip

Archive:  object_detection.zip
   creating: object_detection/
   creating: object_detection/anchor_generators/
  inflating: object_detection/anchor_generators/flexible_grid_anchor_generator.py  
  inflating: object_detection/anchor_generators/flexible_grid_anchor_generator_test.py  
  inflating: object_detection/anchor_generators/grid_anchor_generator.py  
  inflating: object_detection/anchor_generators/grid_anchor_generator_test.py  
  inflating: object_detection/anchor_generators/multiple_grid_anchor_generator.py  
  inflating: object_detection/anchor_generators/multiple_grid_anchor_generator_test.py  
  inflating: object_detection/anchor_generators/multiscale_grid_anchor_generator.py  
  inflating: object_detection/anchor_generators/multiscale_grid_anchor_generator_test.py  
 extracting: object_detection/anchor_generators/__init__.py  
   creating: object_detection/box_coders/
  inflating: object_detection/box_coders/faster_rcnn_box_coder.py  
  inflating: object_detection/box_coder

### Add object detection folder to PYTHONPATH

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

1.15.0


In [0]:
# This is needed since the notebook is stored in the object_detection folder.
import sys
import os
sys.path.append("..")
print(os.listdir("../content"))
from content import object_detection
#print(object_detection)
os.environ['PYTHONPATH'] = ':../content/:../content/slim/:'+os.environ['PYTHONPATH']
print(os.environ['PYTHONPATH'])

['.config', 'training.zip', 'object_detection.zip', 'faster_rcnn_resnet50_coco_2018_01_28.tar.gz', 'object_detection', 'test.record', '.ipynb_checkpoints', 'slim.zip', 'faster_rcnn_resnet50_coco.config', 'train.record', 'slim', 'faster_rcnn_resnet50_coco_2018_01_28', 'drive', 'training', 'sample_data']
:../content/:../content/slim/:/tensorflow-1.15.0/python3.6:/env/python


### Copy your previous checkpoints to colab
Training is the folder where your trained model will be saved

In [0]:
!cp '/content/drive/My Drive/CardID-project/card_detection/training.zip' training.zip

In [0]:
!unzip training.zip

Archive:  training.zip
   creating: training/
  inflating: training/faster_rcnn_resnet50_coco.config  
  inflating: training/labelmap.pbtxt  


### Copy your train and test data to colab

In [0]:
!cp '/content/drive/My Drive/CardID-project/card_detection/test.record' test.record
!cp '/content/drive/My Drive/CardID-project/card_detection/train.record' train.record

### Download pre-trained Faster_RCNN_RESNET50 model
You can download any other pre-trained model from this link just make sure you choose a model with boxes as outputs
https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md


In [0]:
!wget http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet50_coco_2018_01_28.tar.gz

--2020-03-23 14:35:32--  http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet50_coco_2018_01_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.130.128, 2404:6800:4003:c03::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.130.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 381355771 (364M) [application/x-tar]
Saving to: ‘faster_rcnn_resnet50_coco_2018_01_28.tar.gz’

faster_rcnn_resnet5 100%[===================>] 363.69M  57.8MB/s    in 6.8s    

2020-03-23 14:35:40 (53.6 MB/s) - ‘faster_rcnn_resnet50_coco_2018_01_28.tar.gz’ saved [381355771/381355771]



In [0]:
!tar -xzf faster_rcnn_resnet50_coco_2018_01_28.tar.gz

In [0]:
LOG_DIR = 'training'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)


In [0]:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

--2020-03-15 00:58:46--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.233.35.85, 34.238.36.128, 52.207.93.234, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.233.35.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.13M  14.0MB/s    in 0.9s    

2020-03-15 00:58:48 (14.0 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
!pip install numpy==1.17.4

     |████████████████████████████████| 20.0MB 156kB/s 
ERROR: tensorflow-model-optimization 0.2.1 requires enum34~=1.1, which is not installed.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 1.15.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.2
    Uninstalling numpy-1.18.2:
      Successfully uninstalled numpy-1.18.2


### Start the training process

In [0]:
!python /content/object_detection/model_main.py --model_dir=/content/training --pipeline_config_path=/content/faster_rcnn_resnet50_coco.config

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0323 15:30:00.673537 139721268971392 module_wrapper.py:139] From /content/object_detection/utils/config_util.py:102: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0323 15:30:00.676077 139721268971392 module_wrapper.py:139] From /content/object_detection/model_lib.py:628: The name tf.logging.warning is deprecated. Please use tf.compat.v1.logging.warning instead.

W0323 15:30:00.676233 139721268971392 model_lib.py:629] Forced number of epochs for all eval validations to be 1.

W0323 15:30:00.676355 139721268971392 module_wrapper.py:139] From /content/object_detection/utils/config_util.py:488: T

### Upload the trained model to our google drive

In [0]:
!zip -r training.zip training

  adding: training/ (stored 0%)
  adding: training/model.ckpt-0.index (deflated 77%)
  adding: training/model.ckpt-3973.meta (deflated 94%)
  adding: training/eval_0/ (stored 0%)
  adding: training/eval_0/events.out.tfevents.1584978056.de9ecf97bc00 (deflated 44%)
  adding: training/graph.pbtxt (deflated 97%)
  adding: training/checkpoint (deflated 65%)
  adding: training/.ipynb_checkpoints/ (stored 0%)
  adding: training/events.out.tfevents.1584974778.de9ecf97bc00 (deflated 94%)
  adding: training/model.ckpt-3973.data-00000-of-00001 (deflated 11%)
  adding: training/faster_rcnn_resnet50_coco.config (deflated 65%)
  adding: training/model.ckpt-1971.meta (deflated 94%)
  adding: training/events.out.tfevents.1584977424.de9ecf97bc00 (deflated 94%)
  adding: training/model.ckpt-0.meta (deflated 94%)
  adding: training/model.ckpt-3973.index (deflated 72%)
  adding: training/model.ckpt-1971.index (deflated 72%)


In [0]:
!pip install -U -q PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
uploaded = drive.CreateFile({'title': 'training.zip'})
uploaded.SetContentFile('training.zip')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

RedirectMissingLocation: ignored